In [2]:
import os
import base64
import requests
import json
import numpy as np


In [3]:
import os
import json
import time
from tqdm import tqdm
import openai

import concurrent.futures
import random
import json
import time
from collections import Counter
from functools import partial

import requests
from PIL import Image
import base64
import json
import time
from io import BytesIO

import torchvision.transforms.functional as F


In [4]:
tool_list = ["Bipolar electrode",
"Crescent knife",
"Cup forcep",
"Curette",
"Cutting forcep",
"Driller",
"Freer",
"Injector",
"Monopolar electrode",
"Navigation probe",
"Rongeur",
"Scalpel",
"Scissor",
"Suction tube",
"Ultrasound probe"]

In [5]:
openai.api_key='1f6c65a6a30b4a60ae2c377ba944d3a8'
openai.api_base='https://cairllm2.openai.azure.com/'
openai.api_type='azure'
openai.api_version='2023-07-01-preview'
#openai.azure_endpoint = "https://operationllm.openai.azure.com/"
engine="gpt-4-turbo"
azure_endpoint = "https://operationllm.openai.azure.com/"

In [6]:
def ask_gpt(messages, max_retries=35, temperature=0, top_p=0.9, max_tokens=512):
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
    #print(messages)
    for i in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
            engine="gpt-4", # engine = "deployment_name".
                messages=messages
)
            if os.getenv('DEBUG_PRINT'):
                print(response['choices'][0]['message']['content'])
            return response['choices'][0]['message']['content']
        except Exception as e:
            if type(e) in [openai.error.InvalidRequestError, KeyError]:
                print(type(e), e)
                return None
            print(type(e), e)
            time.sleep(2)
            continue    

In [ ]:
def generate_worker(examples,image_dir,info_dict):
    # 1. captions_strs + objects_strs -> questions
    # 2. questions -> grounding dino input
    # 3. grounding dino input -> grounding dino output
    # 4. captions_strs + objects_strs + questions + grounding dino output -> answer

    # 1. captions_strs + objects_strs -> questions
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can analyze a single image. You receive one or two sentences, each describing the same image you are observing.
Generate a question that users may be interested to ask about the image. The question should ask the AI to segment the surgical instrument in the image. The question should be answerable by the given sentences.
The question should ask the AI to detect either some SPECIFIC specific instrument or a collective name like "surgical instrument". Remeber, The question should be answerable by the given sentences."""},
    {"role": "user", "content": examples[0]['captions']+'\n'+examples[0]['objects']},
    {"role": "assistant", "content": examples[0]['question']},
    {"role": "user", "content": examples[1]['captions']+'\n'+examples[1]['objects']},
    {"role": "assistant", "content": examples[1]['question']},
    {"role": "user", "content": info_dict["caption"] + '\n' + info_dict["object"]}
    ]
    #print("messages_1: ",messages)
    question = ask_gpt(messages, temperature=0.01, top_p=0.95)
    if question is None:
        print("question is None, return None")
        return None
    #print(question)
    # 2. questions -> grounding dino input
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can help to extract information from an a sentence. 
You will be given a question about detecting something in an image. Please extract the main object name from the question. Using '.' to concat multiple object names."""},
    {"role": "user", "content": examples[0]['question']},
    {"role": "assistant", "content": examples[0]['detect_input']},
    {"role": "user", "content": examples[1]['question']},
    {"role": "assistant", "content": examples[1]['detect_input']},
    {"role": "user", "content": "Please detect the green car in the image."},
    {"role": "assistant", "content": "green car"},
    {"role": "user", "content": question}
    ]
    detect_input = ask_gpt(messages, temperature=0.01, top_p=0.95)
    if detect_input is None:
        #print("detect_input is None, return None")
        return None
    #print(detect_input)
    # 3. grounding dino input -> grounding dino output
    # get grounding dino output
    
    detect_input = detect_input.lower()
    output_objects = detect_input.split(" . ")
    detect_output = {}
    for i in output_objects:
        if i not in info_dict["detect_output"]:
            print(str(i)+" Not detected")
            detect_output[i] = "Not detected"
        else:
            detect_output[i] = info_dict["detect_output"][i]

    # 4. captions_strs + objects_strs + questions + grounding dino output -> answer
    q_temp = "caption: {cap}\ngrounding dino input: {gdin}\ngrounding dino output: {gdout}\nquestion: {q}\n"
    messages = [
        {'role': 'system', 'content': """You are an AI visual assistant that can analyze a single image. 
You receive a few sentences, each describing the same image you are observing. 
Then you receive the output of the segmentation model, with its corresponding input. The output is a list of objects detected in the image, with their corresponding bounding boxes. These coordinates are in the form of bounding boxes, represented as (x1, y1, x2, y2) with floating numbers ranging from 0 to 1. These values correspond to the top left x, top left y, bottom right x, and bottom right y.
Then you receive the question asked by the user. 
Answer the question based on the given information with your best. 
Do not reveal the input information of the image. DO NOT say that you are given the captions and the objects in the image, JUST answer the question as if you are seeing the image for the first time."""},
        {'role': 'user', 'content': q_temp.format(cap=examples[0]['captions'], gdin=examples[0]['detect_input'], gdout=examples[0]['detect_output'], q=examples[0]['question'])},
        {'role': 'assistant', 'content': examples[0]['answer']},
        {'role': 'user', 'content': q_temp.format(cap=examples[1]['captions'], gdin=examples[1]['detect_input'], gdout=examples[1]['detect_output'], q=examples[1]['question'])},
        {'role': 'assistant', 'content': examples[1]['answer']},
        {'role': 'user', 'content': q_temp.format(cap=info_dict["caption"], gdin=detect_input, gdout=detect_output, q=question)},
    ]
    answer = ask_gpt(messages, temperature=0.01, top_p=0.95)
    if answer is None:
        print("answer is None, return None")
        return None

    # return
    return {
        "unique_id": str(time.time()) + '_' + str(info_dict["image_id"],),
        "image_id": info_dict["image_id"],
        "image_file_name": info_dict['image_file_name'],
        "image_path": os.path.join(image_dir, info_dict['image_file_name']),
        "captions": info_dict["caption"],
        "objects": info_dict["object"],
        "question": question,
        "detect_input": detect_input,
        "detect_output": detect_output,
        "answer": answer,
    }

In [10]:
a = {"a":1,
     "b":[1,2],
     "c":None}
if "x" in a["c"]:
    pass

TypeError: argument of type 'NoneType' is not iterable